#Club Overlap Analysis

#Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime

#Read in the Swipes Data

In [2]:
swipes_f15 = pd.read_csv('stern_only_f16 - Copy1.csv')
swipes_s16 = pd.read_csv('stern_only_s17_updated - Copy.csv')

len(swipes_f15),len(swipes_s16)

(27089, 18780)

#Subset Swipes Data based on Relevant Conditions

In [4]:
swipes_f15 = swipes_f15[swipes_f15['N Number'].notnull()]
swipes_s16 = swipes_s16[swipes_s16['N Number'].notnull()]

In [5]:
total_swipes=swipes_f15.append(swipes_s16, ignore_index=True)

In [6]:
total_swipes.to_csv('TotalSwipes1617.csv')

In [7]:
total_swipes = pd.read_csv('TotalSwipes1617.csv')

#Aggregation and Unstacking to get swipes per student per club

In [8]:
f15_hf = total_swipes.groupby(['N Number','Portal']).size()
#print f14_gender
f15_hfs=f15_hf.unstack(level='Portal')
#print f16_gsu
f15_hfs.to_csv('HFS_Total_1617.csv')

In [9]:
total_club_participation = pd.ExcelWriter('Total Club Participation 1617.xlsx', engine='xlsxwriter')
f15_hfs.to_excel(total_club_participation, sheet_name='Year club participation')
total_club_participation.save()

#Similar Analysis but with Unique Student - Club Interaction

In [11]:
f15_nodup=total_swipes.drop_duplicates(subset=['N Number', 'Portal'])

In [12]:
f15_hfu = f15_nodup.groupby(['N Number','Portal']).size()
f15_hfud=f15_hfu.unstack(level='N Number')
f15_hfud.to_csv('HFS_Unique_1617.csv')

In [13]:
unique_club_participation = pd.ExcelWriter('Unique Club Participation 1617.xlsx', engine='xlsxwriter')
f15_hfud.to_excel(unique_club_participation, sheet_name='Unique Year club participation')
unique_club_participation.save()

#Cosine Similarity

In [14]:
fall = pd.read_csv('HFS_Unique_1617.csv')

In [15]:
fall.name='total'

In [16]:
def compute_similarity(df,file_name):
    
    columns = list(df.columns.values)
    df.rename(columns={'Portal':'Club'},inplace=True)
    
    df = df.set_index('Club')
    for i in df.index.values:
        print (i)
        for j in columns[3:]:
            x11 = np.array(df.loc[i,j])
            x22 = np.array(df.loc[i,columns[2]])
            x12 = np.divide(x11,x22)
            df.loc[i,j] = x12

    df = df.reset_index()
    df.fillna(0,inplace=True)
    df.to_csv(file_name+' unit vector_1.csv')
    unit_vector = df.set_index('Club')
    imp = columns[3:]
    length=len(imp)
    print(length)
    clubs_df = pd.DataFrame()
    clubs_df['clubs'] = list(unit_vector.index.values)
    clubs_df = clubs_df.set_index('clubs')
    #q=20
    for i in clubs_df.index.values:
        print (i)
        for j in clubs_df.index.values:

            x1 = np.array(unit_vector.loc[i,imp])
            x2 = np.array(unit_vector.loc[j,imp])
            x = np.sum(np.multiply(x1,x2))
            clubs_df.loc[i,j] = x
    clubs_df = clubs_df.reset_index()
    clubs_df.to_csv(file_name +' clubs cosine similarity.csv')